In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

## Data Loading and Preprocessing

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')

train.head()

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtypes
        
        if col_type in numerics:
            col_min = df[col].min()
            col_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if col_min > np.iinfo(np.int8).min and col_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif col_min > np.iinfo(np.int16).min and col_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif col_min > np.iinfo(np.int32).min and col_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif col_min > np.iinfo(np.int64).min and col_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if col_min > np.finfo(np.float32).min and col_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    
    end_mem = df.memory_usage().sum() / 1024**2
    
    if verbose:
        print('Mem. usage decreased to {:.2f}Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
train = reduce_memory_usage(train)
test = reduce_memory_usage(test)

In [ ]:
train_x = train.drop(['Id', 'Cover_Type'], axis = 1)
test_x = test.drop('Id', axis = 1)

y_data = train.Cover_Type

In [ ]:
stdsc = StandardScaler()
train_x = stdsc.fit_transform(train_x)
test_x = stdsc.transform(test_x)

encoder = OneHotEncoder()
y_data = encoder.fit_transform(y_data.values[:, np.newaxis]).toarray()

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(train_x, y_data)

## Modeling

In [ ]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(54,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(7, activation='softmax')
])

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics = ['acc'])

In [ ]:
checkpoint_path = 'tmp_checkpoint.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                            save_weights_only = True,
                            save_best_only = True,
                            monitor = 'loss',
                            verbose = 0)

In [ ]:
BATCH_SIZE = 512 
EPOCHS = 15

model.fit(x_train, y_train, batch_size = BATCH_SIZE, epochs = EPOCHS, callbacks = [checkpoint])

model.load_weights(checkpoint_path)

## Validation

In [ ]:
y_pred = model.predict(x_valid)
accuracy_score(y_valid.argmax(axis = 1), y_pred.argmax(axis = 1))

In [ ]:
y_test = model.predict(test_x)
y_test = y_test.argmax(axis = 1).reshape(-1, ) + 1

submission.Cover_Type = y_test
submission.to_csv('submission.csv', index = False)
print("Done")